## Here 1

In [1]:
sample = "[16.22, 84.86, 36.76, 52.43, 69.19, 45.95, 108.11, 50.27, 118.92, 60.0, 134.05, 76.22, 132.97, 104.32, 145.95, 128.11, 151.35, 143.24, 143.78, 164.86, 139.46, 174.59, 151.35, 191.89, 189.19, 223.24, 230.27, 265.41, 235.68, 268.65, 238.92, 282.7, 232.43, 297.84, 223.78, 302.16, 183.78, 290.27, 158.92, 271.89, 105.95, 251.35, 122.16, 272.97, 137.3, 287.03, 149.19, 312.97, 162.16, 338.92, 169.73, 349.73, 174.05, 361.62, 161.08, 396.22, 149.19, 423.24, 139.46, 437.3, 125.41, 448.11, 113.51, 453.51, 104.86, 454.59, 98.38, 444.86, 78.92, 426.49, 55.14, 402.7, 40.0, 387.57, 21.62, 368.11, 1.08, 348.65, 0.0, 136.76, 0.0, 123.78]"

In [2]:
sample = sample.replace('[', '').replace(']', '').split(', ')

In [3]:
sample = list(map(lambda i: float(i), sample))

In [4]:
sample = [round(sample[i]/640, 3) if i%2!=0 else round(sample[i]/480, 3) for i in range(len(sample))]

In [5]:
sample = str(sample)

In [6]:
from transformers import AutoModelForVision2Seq, AutoTokenizer, AutoProcessor
import torch

model_name = "HuggingFaceTB/SmolVLM-256M-Instruct"
model = AutoModelForVision2Seq.from_pretrained(model_name, torch_dtype=torch.float16).to("cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name)

In [7]:
import numpy as np

# Generate the float numbers from 0.00 to 1.00 with a step of 0.01
float_numbers = np.arange(0.00, 1.001, 0.001)
precision = 0.001
# Convert the float numbers to strings with two decimal places
new_tokens = [f"{num:.{len(str(precision).split('.')[-1])}f}" for num in float_numbers]

# Assuming 'processor' is an object that has a 'tokenizer' attribute
# and that 'tokenizer' has an 'add_tokens' method (common in libraries like Hugging Face Transformers)
processor.tokenizer.add_tokens(new_tokens)
processor.tokenizer.add_tokens([', '])

1

In [8]:
sample

'[0.034, 0.133, 0.077, 0.082, 0.144, 0.072, 0.225, 0.079, 0.248, 0.094, 0.279, 0.119, 0.277, 0.163, 0.304, 0.2, 0.315, 0.224, 0.3, 0.258, 0.291, 0.273, 0.315, 0.3, 0.394, 0.349, 0.48, 0.415, 0.491, 0.42, 0.498, 0.442, 0.484, 0.465, 0.466, 0.472, 0.383, 0.454, 0.331, 0.425, 0.221, 0.393, 0.255, 0.427, 0.286, 0.448, 0.311, 0.489, 0.338, 0.53, 0.354, 0.546, 0.363, 0.565, 0.336, 0.619, 0.311, 0.661, 0.291, 0.683, 0.261, 0.7, 0.236, 0.709, 0.218, 0.71, 0.205, 0.695, 0.164, 0.666, 0.115, 0.629, 0.083, 0.606, 0.045, 0.575, 0.002, 0.545, 0.0, 0.214, 0.0, 0.193]'

In [9]:
processor.tokenizer.encode('[')

[75]

In [10]:
len(processor.tokenizer.encode(sample))

189

In [11]:
processor.tokenizer.encode('[0.034, 0.133, 0.077]')

[75, 49314, 50281, 49413, 50281, 49357, 77]

In [12]:
model.lm_head

Linear(in_features=576, out_features=49280, bias=False)

In [13]:
model.resize_token_embeddings(len(processor.tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50282, 576, padding_idx=2)

In [14]:
model.lm_head

Linear(in_features=576, out_features=50282, bias=False)

## Here 2

In [1]:
from transformers import AutoModelForVision2Seq, AutoTokenizer, AutoProcessor
import torch

model_name = "HuggingFaceTB/SmolVLM-256M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
processor = AutoProcessor.from_pretrained(model_name)

In [4]:
from transformers.image_utils import load_image

In [8]:
processor.apply_chat_template([
    {
        'role': 'user',
        'content': [
            {'type': 'image', 'image': [load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")]},
            {'type': 'text', 'text': 'This is some path.'}
        ]
    }
], tokenize=False)

'<|im_start|>User:<image>This is some path.<end_of_utterance>\n'

In [5]:
image = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")

In [7]:
type(image)

PIL.Image.Image

In [13]:
import datasets

refcocog_dataset = datasets.load_dataset("jxu124/refcocog", split="train")

In [15]:
type(refcocog_dataset[:5])

dict

In [11]:
refcocog_dataset['sentences']

[[{'raw': 'Two woman one in black eatting and the other has a white shirt at the desk',
   'sent': 'two woman one in black eatting and the other has a white shirt at the desk',
   'sent_id': 0,
   'tokens': ['two',
    'woman',
    'one',
    'in',
    'black',
    'eatting',
    'and',
    'the',
    'other',
    'has',
    'a',
    'white',
    'shirt',
    'at',
    'the',
    'desk']},
  {'raw': 'Woman in white shirt looking down at laptop computer.',
   'sent': 'woman in white shirt looking down at laptop computer',
   'sent_id': 1,
   'tokens': ['woman',
    'in',
    'white',
    'shirt',
    'looking',
    'down',
    'at',
    'laptop',
    'computer']}],
 [{'raw': 'A TV with a woman being interviewed on it.',
   'sent': 'a tv with a woman being interviewed on it',
   'sent_id': 4,
   'tokens': ['a',
    'tv',
    'with',
    'a',
    'woman',
    'being',
    'interviewed',
    'on',
    'it']},
  {'raw': 'A woman with sunglasses on her head on the television being interviewe